In [ ]:
%pylab inline
import seaborn as sns
import xarray as xr
import gnl.xarray
from gnl.xarray import xcorr, xr2mat

from sklearn.linear_model import Ridge

# Re-presentation of last weeks results

## Autocorrelation of precipitation

In [ ]:
precip = xr.open_dataset("/home/disk/eos4/nbren12/Data/id/0c31ce4c6763d8ec2abccafe6bde2fa0bed8124d/data/2d/Prec.nc")\
           .Prec\
           .coarsen(x=64)\
           .compute()

In [ ]:
precip.plot()

In [ ]:
ac = xcorr(precip, dim='time').mean('x')
plt.plot(ac.time * 24, ac)
axhline(1/exp(1))
xlim([0,24])
plt.xlabel("lag [hr]")
plt.ylabel("Correlation")

The precipitation in this 2D dataset decorrelates much more rapidly than in the NGAqua dataset.

In [ ]:
import os

ngaqua = "/home/disk/eos4/nbren12/Prec_coarse.nc"
prec_ngaqua = xr.open_dataset(ngaqua).Prec

def f(x):
#     print(x)
    return xcorr(x, dim='time').mean('x')

acf = prec_ngaqua.groupby('y').apply(f)

plt.pcolormesh(acf.time*24, acf.y/1e3, acf.T, cmap='viridis')
plt.colorbar()
lev = 1/exp(1)
cs = plt.contour(acf.time*24, acf.y/1e3, acf.T, [lev])

plt.text(6.0, 5800, u"$e^{-1}$", color="w")

xlim([0,20])
plt.ylabel('y [km]')
plt.xlabel('Lag [hr]')
plt.title(u"Autocorrelation of 160 km averaged precip")

The autocorrelationt time in the NGAqua simulations is a little bit longer.

## Q1 and Q2

In [ ]:
p = xr.open_dataset("wd/stat.nc").p
q1 = xr.open_dataset("wd/calc/q1.nc")
q2 = xr.open_dataset("wd/calc/q2.nc").q2

### Mean

In [ ]:
figure(figsize=(3,3*1.61))
mu = q1.q1.mean(['x','time'])
plt.plot(mu, p, label='Q1')

mu = q2.mean(['x','time'])
plt.plot(mu, p, label='Q2')


plt.plot(q1.tend.mean('time'), p, label='$Q_r$')
ylim([1000, 10])
plt.legend()
xlabel("K/day")
ylabel("p [hPa]")

Here are the plots of the domain and time mean Q1 and Q2, where Q1 does not include the radiative component and the large-scale forcings.

### Variance

Here the standard deviation of Q1 and Q2 for each vertical level.

In [ ]:
figure(figsize=(3,3*1.61))
mu = q1.q1.std(['x','time'])
plt.plot(mu, p, label='Q1')

mu = q2.std(['x','time'])
plt.plot(mu, p, label='Q2')


ylim([1000, 10])
plt.legend()
xlabel("K/day")
ylabel("p [hPa]")

## Maximum Covariance Analysis

I normalizing using the mass-weighted standard deviation for each variable, because I cannot think of a more parsimonous way to do this. I do not include LHF and SHF in the analysis beacuse it has different physical units than $s_l$ and $q_t$, which can each be put into J/kg. This means that sort of ad-hoc scaling must be used to weight LHF equally to a whole profile of $q_t$ and $s_l$. Since the mass-weighted integrals Q1 and Q2 each of units of W/m2,  the LHF and SHF could be modelled as part of the respones, but this would violate our intuition that LHF can control convection.

In [ ]:
def plot_spec(eig):
    plt.bar(np.r_[:20],np.cumsum(eig.values)*100)
    plt.xticks(np.r_[:20])
    plt.ylabel("Cummulative (Co)variance Explained")
    plt.xlabel("Number of modes")

def plot_modes(d, modes):

    colors = ["blue", "dark red", "medium green", "dark blue"]
    sns.set_palette(sns.xkcd_palette(colors))

    fig, axs = plt.subplots(1,4, sharey=True)

    # keep lines for making figure
    lines = []
    for m in modes:
        dm = d.isel(m=m)
        
        scal = np.sign(float(dm.qt.sel(z=2e3, method='nearest'))) * np.abs(dm.qt).max()

        lines.append((axs[0].plot(dm.q1c/scal,d.p)[0], f"m = {m}"))
        axs[1].plot(dm.q2/scal,d.p)
        axs[2].plot(dm.sl/scal,d.p)
        axs[3].plot(dm.qt/scal,d.p)
        plt.ylim([1000, 10])

        for ax, title in zip(axs.flat, [r'$Q_1$ [K/day]', r'$Q_2$ [K/day]', r'$s_l$ [K]', r'$q_t$ [g/kg]']):
            ax.set_title(title)


    axs[0].set_ylabel('p [hPa]')        
    fig.legend(*zip(*lines),loc='center right')

With phyiscal units the data look like this.

In [ ]:
sl  =xr.open_dataarray("wd/calc/sl.nc")
qt = xr.open_dataarray("wd/calc/qt.nc")
D = xr.merge([sl, qt])



In [ ]:
fig, (a,b, c) = plt.subplots(1,3, sharey=True)


sample_dims = ['x', 'time']

a.plot(sl.mean(sample_dims), p)
a.set_ylim([1000,10])
a.set_xlabel('$s_l$ [K]')
a.set_ylabel('p [hPa]')


b.plot(qt.mean(sample_dims), p)
b.set_ylim([1000,10])
b.set_xlabel('$q_T$ [g/kg]')

c.plot(qt.std(sample_dims), p, label='SD($q_T$) [g/kg]')
c.plot(sl.std(sample_dims)/5, p, label='SD(s_l)/5 [K]')
c.legend()




The fact that the variance of $q_t$ is concentrated in the lower troposphere could potentially mask some important effects. On the other hand, mid-tropospheric moisture is well understood to be an important control on convection.

In [ ]:
mca = xr.open_dataset("wd/calc/mca.nc")
mca['p'] = p

In [ ]:
plot_spec(mca.eig)

In [ ]:
plot_modes(mca, range(2))

The ratio of magnitude of the first three s_l and q_t modes is around 10, which to me says that the mode is more senstive to small changes in $q_t$. This analysis cannot reveal anything about causation. Note that because these are linear modes, the sign and magnitude is arbitrary. In other words, the ratio the same mode between variables is important. I have scaled all the modes so that moisture is postive in the boundary layer, and the maximum magnitude of $q_t$ is set to 1.

The moisture and apparent heating modes appear to have the standard baroclinic mode interpretations, whereas the structure of the temperature mode is more complex. This analysis could imply that convection primarily responds to the particular vertical structure of moisture. The basis which captures vertical shifts in the moisture profile the best, is simply a Fourier mode type basis, so I believe that is what we are seeing here.

# Partial Least Squares Analysis

PLS is similiar to maximum covariance analysis. If $X$ has size $s \times  n$ and $Y$ has sign $s \times m$, then the data can be decomposed 
$$ X = TP' + E,\quad Y= UQ' +F.$$
To understand the PLS algorithm, we first consider the problem of finding the pair of weights with unit norm $u$ and $v$ which maximize the quantity $cov(Xu, Yv)=u'X'Yv$. These weights are just the left and right eigenvectors of the cross-covariance matrix $X'Y$, which can be found using a power iteration computation. Once the first maximizing weight is found, the matrix is deflated

## 2D Homogenous Data

In [ ]:
from pls_plots import plot_coef, plot_pls_mode, plot_coef1, pls_plots

Open the partial least squares analysis

In [ ]:
pls_plots("wd/calc/pls.nc", "wd/stat.nc")

Note that because the 2D SAM simulation has fixed radiative cooling, the demeaned $Q_1$ is the same as the demeaned $Q_{1c} = Q_1 -Q_r$.

## NGAqua at equator

In [ ]:
pls_plots("wd/ngaqua/pls.nc", "wd/ngaqua/p.nc")

### Estimated Linear Response function

This is given by $PT'UQ'$

Here is the coefficient matrix for the SAM run. It looks pretty reasonable compared to the linear response functions generated by Zhiming Kuang, but I need to make a more careful comparison.

In [ ]:
ax = plot_coef("wd/calc/pls.pkl", "wd/w.nc")
ax[0,0].set_xlim([0,15000])

While the loading matrices $P$ and $Q$ are vey similar between the two datasets. I am getting weird results for the coefficient matrix of the NGAqua data. My guess is that this is caused by using the daily averaged data.

In [ ]:
plot_coef("wd/ngaqua/pls.pkl", "wd/ngaqua/w.nc");

# Penalization approaches
In general, MCA can be viewed as basically a regularization approach to fitting a linear model 
$$Y = XB$$
where $Y$ is a $s\times n$ matrix and $X$ is an $s\times m$ matrix. $s$ is the number of samples (e.g. grid boxes and time points) while $m$ and $n$ are the number of feature in the output and input respectively. For example, the features of the input in the cumulus parametrization problem are the vertical profiles of $s_l$ and $q_t$ evaulated at each vertical grid point. For the 2D SAM dataset this implies that $m=n=n_z\times 2=128$ data points.

The standard way to fit a multivariate linear regression problem is to perform a least squares fit. However, when performing a linear fit, the condition number of the problem should be small. Is this the case?

In [ ]:
D = xr.open_mfdataset(["wd/calc/sl.nc", "wd/calc/qt.nc", "wd/calc/q1.nc", "wd/calc/q2.nc"])
w = xr.open_dataarray("wd/w.nc")

D = D * w
X,scalex = xr2mat(D[['qt', 'sl']], ['x', 'time'], ['z'], scale=True)

# perform computation
X.load()

# compute condition number
np.linalg.cond(X)

As you can see, the condition number of this matrix is extremely high, so using ordinary least squares to find the matrix $B$ is a hopeless task. The reason the matrix $X$ is ill-condition is beacause of collinearity in the input variables. For example, $s_l$ and $q_t$ at neighboring heights will be highly correlated.

This ill conditioning is very common in multivariate problems like this, and there are a variety of popular techinques that are used to "regularize" the problem. Some examples of these techniques include

1. Penalization methods like Ridge Regression (L2 penalty) and the LASSO (L1 penalty). These methods penalize large coefficent values, and the LASSO promotes sparsity. Another simple idea, which would promote smooth solutions would be to the 2nd vertical derivative of the fitted coefficients.
2. Partial least squares techniques. This class of problems includes Maximum Covariance Analysis (MCA) as well as the namesake Partial Least Squares regression technique. These methods essentially project both the inputs and outputs onto truncated sets of orthogonal modes which covary strongly. This method improves the conditionining of the problem by projecting the data to a particularly efficient basis.

As a first step towards finding the whole linear response function, we attempt to find a linear model for the precipitation, which is related to the vertical integral of the first mode we found in the MCA analysis.

In [ ]:
# open and subsample precipitation dataset
prec = xr.open_dataarray("wd/A64/2d/Prec.nc").reindex_like(D)

y = xr2mat([prec], ['x','time'], [], scale=False)[0]

Unfortunately, precipitation is very fat-tailed as can be seen in the following log-pdf plot.

In [ ]:
from gnl.plots import loghist

loghist(prec.values.ravel())
ylim([-10,0])
legend(["log-pdf of precip", "log-pdf of gaussian"])
xlabel("mm/day")

Because precipitation is very fat-tailed we transform it using the logarithm  so that 
$$y = \log(P + .1) $$.

In [ ]:
eps = .1
yt = np.log(y + eps)

The resulting histogram plot looks like this:

In [ ]:
loghist(yt.values.ravel())
legend(["log-pdf of y", "gaussian comparison"])

This distribution is clearly not normalized, but it is better than before. Now, let's try to fit a ridge regression model.

## Ridge Regression

Ridge regression is given by the following penalized least-squares problem 
$$\min_{B} \frac{1}{2}|Y - XB|^2 + \alpha |B|^2.$$
The parameter $\alpha$ is used to penalize large values of the estimated coefficients $B$.

In terms of physical quantities, the minimization problem I aim to solve is 
$$\min_{a,b} \frac{1}{2}\left|\tilde{P} + a(z)\tilde{s_l}(z)  + b(z) \tilde{q_t}(z)\right|^2 + \alpha \int \left[a(z)^2   + b(z)^2 \right] \rho_0 dz$$.
Where $\tilde{\cdot}$ is the normalization operator. The profiles $q_t$ and $s_l$ are normalized by the square root of the vertically integrated variance. 

In [ ]:
def plot_ridge(alpha):
    mod = Ridge(alpha=alpha, normalize=False)
    mod.fit(X-X.mean('samples'), yt)
    
    Bqt, Bsl = np.split(mod.coef_.ravel(), 2)
    
    plt.subplot(121)
    plt.plot(Bqt/w, p)
    plt.xlabel(r"$q_l$")
    plt.ylim([1000,10])
    plt.ylabel('p [hPa]')
    
    plt.subplot(122)
    plt.plot(Bsl/w, p)
    plt.xlabel(r"$s_l$")
    plt.ylim([1000,10])
    
    plt.suptitle(f"Coefficients for alpha = {alpha}")
    
    # plot precip output
    plt.figure(figsize=(6/1.61,6))
    pred = np.exp(xr.DataArray(mod.predict(X-X.mean('samples')), yt.coords).unstack('samples'))-eps
    pred.plot(vmin=0, x='x', y='time')
    plt.title(f"Predicted Precipitation for alpha = {alpha}")

As you can see, for small $\alpha$ the solutions are not very stable:

In [ ]:
plot_ridge(.1)

But increasing $\alpha$ leads to much more stable answers.

In [ ]:
plot_ridge(1000.0)

The standard practice in machine learning is to choose parameters like $\alpha$ can be chosen by a K-fold cross-validation. I would recommend using the relative entropy between the distributions of the actual and predicted precipitation as a measure of the performance of given set of hyper parameters.